In [1]:
# Gee's STV implementation for shuffled voting
# adapted from https://gist.github.com/c2huc2hu/a5826b52e838c5845dca
import csv
votename = 'FTC19' # the base name that the code will search for when loading the two csvs

# load vote .csv into data
vote_file = open(votename +".csv", 'r')
raw_data = vote_file.readlines() # raw_data is a list of all the rows in the .csv
vote_file.close()

# load a .csv containing all candidates
file = open(votename+"_candidates.csv",'r')
candidate_file = file.readlines()
file.close()

raw_data = raw_data[1:] # shave off the top row, as it contains the headings

# create a list with all candidates
newloop = True
for line in candidate_file:
    cells = line.split(',')
    if newloop == True:
        candidates = [cells[0].rstrip('\n')] # .rstrip('\n') shaves off the newline marker from the row
        newloop = False
    else:
        candidates.append(cells[0].rstrip('\n'))

# generate usable ballots from raw_data, line by line
newloop = True
votenumber = 0
for line in raw_data: # loop through each item in the raw_data list, which was a line in .csv
    shaved_line = line.rstrip('\n') # since the line is a string, we can shave \n off here
    vote = shaved_line.split(',') # split the string into a list based on where commas are
    
    # initialise each row of the .csv as a list in the ballots list
    if newloop == True: # if this is the first line, then make 
        ballots = [[vote[0]]] # vote[0] is the timestamp!
        newloop = False
    else:
        ballots.append([vote[0]]) # add a list into the ballots list
    
    # convert raw preferences into team names in order of preference
    for i in range(len(candidates)):
        find = 'P' + str(i+1) # denote which preference string to look for
        if find in vote:
            idx = vote.index(find) # return the index of that Preference
            # append candidates in order of preference
            ballots[votenumber].append(candidates[idx-1]) # idx-1 because the vote has timestamp but candidate list doesn't
            pass
        pass
    votenumber += 1

# generate the options dict from list of candidates
options = {}
for candidate in candidates:
    options[candidate] = 0
    
# generate dictionary for distribution tracker that'll export to .csv
tracker = {}
for candidate in candidates:
    tracker[candidate]= []

# the actual counting loop
while(len(options) >= 2):
    for vote in ballots: # loop through all votes
        for preference in vote: # read down the preferences in that vote
            if preference in options: # if the preference is in options, +1 to that option and stop reading that vote
                options[preference] += 1
                break  
    
    # input data to distribution tracker
    for candidate in candidates:
        if candidate in options:
            tracker[candidate].append(options[candidate]) # append number of votes candidate has
    print(options)

    # remove the least popular option(s)
    fewest_votes = min(options.values()) # finds the lowest number of votes
    options_copy = dict(options)
    for i in options:
        if options[i] == fewest_votes:
            del options_copy[i] 
    options = options_copy

    # Reset votes to 0
    for i in options:
        options[i] = 0
    # loop will now start over

# write the .csv with votes for each round
with open(votename + '_outcome.csv', 'w',newline='') as file:
    for candidate in candidates:
        line = [candidate]
        for value in tracker[candidate]:
            line.append(value)
        wr = csv.writer(file, dialect='excel')
        wr.writerow(line)

{'The Bitter Blues': 199, "Blazin' Balls": 200, 'The Clowns': 217, 'Hornets': 360, 'Knights of the Round': 212, 'Red Pandas': 203, 'Turtle Sliders': 261, 'Woodland Foxes': 168}
{'The Bitter Blues': 211, "Blazin' Balls": 212, 'The Clowns': 232, 'Hornets': 390, 'Knights of the Round': 234, 'Red Pandas': 254, 'Turtle Sliders': 287}
{"Blazin' Balls": 265, 'The Clowns': 255, 'Hornets': 421, 'Knights of the Round': 263, 'Red Pandas': 284, 'Turtle Sliders': 328}
{"Blazin' Balls": 309, 'Hornets': 500, 'Knights of the Round': 296, 'Red Pandas': 329, 'Turtle Sliders': 370}
{"Blazin' Balls": 374, 'Hornets': 578, 'Red Pandas': 398, 'Turtle Sliders': 444}
{'Hornets': 730, 'Red Pandas': 500, 'Turtle Sliders': 540}
{'Hornets': 945, 'Turtle Sliders': 781}
